#### QandA implementation using haystack
---


In [2]:
from haystack import Finder
from haystack.indexing.cleaning import clean_wiki_text
from haystack.indexing.io import write_documents_to_db, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

In [5]:
#Elasicsearch config details
#host_ = "localhost"
#user_ = ""
#pass_ = ""

### Cloud
host_ = "5f6d021c98ed4175b7ef84ecf7b60b78.asia-south1.gcp.elastic-cloud.com:9243"
user_ = "elastic"
pass_ = "Imr3WP20gEbPvQ5a6jDpAA7d"

In [6]:
# Connect to Elasticsearch

from haystack.database.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host=host_, username=user_, password=pass_, index="document")

# In-Memory Document Store
#from haystack.database.memory import InMemoryDocumentStore
#document_store = InMemoryDocumentStore()

# SQLite Document Store
#from haystack.database.sql import SQLDocumentStore
##db_ = "sqlite:///" + "/home/krishna/qa.db"
#document_store = SQLDocumentStore(url=db_)

05/04/2020 19:46:07 - WARNING - elasticsearch -   PUT http://5f6d021c98ed4175b7ef84ecf7b60b78.asia-south1.gcp.elastic-cloud.com:9243:9200/document [status:N/A request:0.003s]
Traceback (most recent call last):
  File "/home/krishna/anaconda3/envs/ktrain/lib/python3.6/site-packages/urllib3/connection.py", line 160, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/home/krishna/anaconda3/envs/ktrain/lib/python3.6/site-packages/urllib3/util/connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/krishna/anaconda3/envs/ktrain/lib/python3.6/socket.py", line 745, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/krishna/anaconda3/envs/ktrain/lib/python3.6/site-packages/elast

05/04/2020 19:46:07 - WARNING - elasticsearch -   PUT http://5f6d021c98ed4175b7ef84ecf7b60b78.asia-south1.gcp.elastic-cloud.com:9243:9200/document [status:N/A request:0.002s]
Traceback (most recent call last):
  File "/home/krishna/anaconda3/envs/ktrain/lib/python3.6/site-packages/urllib3/connection.py", line 160, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/home/krishna/anaconda3/envs/ktrain/lib/python3.6/site-packages/urllib3/util/connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/krishna/anaconda3/envs/ktrain/lib/python3.6/socket.py", line 745, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/krishna/anaconda3/envs/ktrain/lib/python3.6/site-packages/elast

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7fe0e88d8828>: Failed to establish a new connection: [Errno -2] Name or service not known) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7fe0e88d8828>: Failed to establish a new connection: [Errno -2] Name or service not known)

In [4]:
doc_dir = '/home/krishna/projects/haystack/docs/'

# Now, let's write the docs to our DB.
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
write_documents_to_db(document_store=document_store, document_dir=doc_dir, \
                      only_empty_db=True, split_paragraphs=True)

05/03/2020 13:35:42 - INFO - elasticsearch -   POST http://localhost:9200/_count [status:200 request:0.083s]
05/03/2020 13:41:40 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:1.657s]
05/03/2020 13:41:42 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:1.500s]
05/03/2020 13:41:44 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:1.313s]
05/03/2020 13:41:45 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:1.465s]
05/03/2020 13:41:47 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:1.342s]
05/03/2020 13:41:48 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:1.328s]
05/03/2020 13:41:50 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:1.463s]
05/03/2020 13:41:51 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:1.326s]
05/03/2020 13:41:53 - INFO 

05/03/2020 13:43:47 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:1.130s]
05/03/2020 13:43:49 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:1.270s]
05/03/2020 13:43:53 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:3.506s]
05/03/2020 13:43:54 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:1.343s]
05/03/2020 13:43:56 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:1.361s]
05/03/2020 13:43:58 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:1.500s]
05/03/2020 13:43:59 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:1.632s]
05/03/2020 13:44:01 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:1.169s]
05/03/2020 13:44:03 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:1.928s]
05/03/2020 13:44:04 - INFO -

In [4]:
## Using BM25
from haystack.retriever.elasticsearch import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

# An in-memory TfidfRetriever based on Pandas dataframes

#from haystack.retriever.tfidf import TfidfRetriever
#retriever = TfidfRetriever(document_store=document_store)

In [10]:
### Reader

#reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=False)
reader = FARMReader(model_name_or_path='deepset/roberta-base-squad2-covid', use_gpu=False)
reader.save("data/roberta")
reader_2 = FARMReader(model_name_or_path='data/roberta', use_gpu=False)
#reader = TransformersReader(model="deepset/roberta-base-squad2",tokenizer="deepset/roberta-base-squad2-covid", use_gpu=-1)

05/04/2020 17:09:21 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
05/04/2020 17:09:21 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2-covid` locally. Try to download from model hub ...
05/04/2020 17:09:26 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
05/04/2020 17:09:32 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"loss_ignore_index": -1}
05/04/2020 17:09:32 - WARNING - farm.modeling.adaptive_model -   ML logging didn't work: Yaml file '/home/krishna/projects/haystack/notebooks/mlruns/0/meta.yaml' does not exist.
05/04/2020 17:09:39 - WARNING - farm.data_handler.processor -   ML logging didn't work: Yaml file '/home/kr

In [16]:
finder = Finder(reader, retriever)

In [17]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers.
prediction = finder.get_answers(question="what is Covid-19?", top_k_retriever=5, top_k_reader=5)

05/03/2020 15:01:57 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.110s]
05/03/2020 15:01:57 - INFO - haystack.retriever.elasticsearch -   Got 5 candidates from retriever
05/03/2020 15:01:57 - INFO - haystack.finder -   Reader is looking for detailed answer in 108615 chars ...


In [18]:
results = print_answers(prediction, details="all")

{   'answers': [   {   'answer': 'a contagious respiratory illness',
                       'context': 'Coronavirus (others include SARS, H5N1, '
                                  'H1N1 and MERS), is a contagious respiratory '
                                  'illness transmitted through the eyes, nose, '
                                  'and mouth, via droplet',
                       'document_id': 'ZWKZ2XEBjtCb_1IEJ6my',
                       'meta': {   'name': '38b7a91d22d77b76c5d268c3943efbf3c4cc3cac.txt'},
                       'offset_end': 92,
                       'offset_end_in_doc': 4366,
                       'offset_start': 59,
                       'offset_start_in_doc': 4333,
                       'probability': 0.8821277386576323,
                       'score': 16.101882934570312},
                   {   'answer': 'a lower airway disease',
                       'context': ' I pay more attention to when I screen a '
                                  'patient?\